In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

#!python -m spacy download pt

In [2]:
data = pd.read_json(
    "/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/ml_model/data/data.json"
).reset_index(drop=True)

In [3]:
data["description"] = data["description"].apply(lambda x: x.lower())

In [4]:
nlp = spacy.load("pt_core_news_sm")

/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/venv/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'pt_core_news_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
def remove_stopwords(text):
    doc = nlp(text)
    filtered_words = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered_words)

In [6]:
data["description"] = data["description"].apply(remove_stopwords)

In [7]:
# Instantiate the TF-IDF vectorizer
vectorizer = TfidfVectorizer(lowercase=True, strip_accents="unicode")

# Apply TF-IDF on the text dataset
tfidf_matrix = vectorizer.fit_transform(data["description"])

# Get the feature names
feature_names = vectorizer.get_feature_names_out()

# Create a new dataframe with the TF-IDF features
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

In [8]:
# Exemplo de entrada do usuário
user_input = "Estou com alguns problemas com meu seguro."

user_input = remove_stopwords(user_input)

In [9]:
# Vetorização do input do usuário
input_vector = vectorizer.transform([user_input])

# Cálculo da similaridade de cosseno entre o input do usuário e cada descrição
similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

In [10]:
most_similar_index = similarity_scores.argmax()

In [11]:
print(
    f"Matriz de similaridade {similarity_scores}\n",
    f"Indice do vetor: {most_similar_index}",
)

Matriz de similaridade [[0.12919409 0.         0.11114526 0.12393859 0.26820331 0.13189763]]
 Indice do vetor: 4


In [12]:
# Resultado
most_similar_description = data["description"][most_similar_index]
print("Descrição mais similar:", most_similar_description)

Descrição mais similar: dúvidas cobertura : cliente dúvidas situações cobertas seguro . precisa esclarecimentos termos condições apólice , exemplos eventos cobertos seguro .


In [13]:
data

,id,description
0,1,problema reembolso : cliente dificuldades obte...
1,2,sinistro veículo : cliente sofreu acidente trâ...
2,3,alteração dados pessoais : cliente mudou ender...
3,4,renovação apólice : cliente apólice seguro pre...
4,5,dúvidas cobertura : cliente dúvidas situações ...
5,6,cancelamento apólice : cliente deseja cancelar...


# Script

In [14]:
data_script = pd.read_json(
    "/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/ml_model/data/data_script.json"
).reset_index(drop=True)

In [81]:
response = data_script[data_script["id"] == most_similar_index]["script"].to_dict()

In [80]:

# class Bot:
#     @router.post("/bot")
#     async def message(connection_string: Depends(DB_URL)):
#         message_history = MongoDBChatMessageHistory(
#             connection_string=connection_string, session_id="test-session"
#         )

#         message_history.add_user_message("hi!")

#         message_history.add_ai_message("whats up?")

'Olá! Bem-vindo ao nosso suporte. Como posso ajudar você hoje?'